<h2>About this Exercise</h2>
<p>In the preceding activity, you derived a Euclidean distance matrix. Now that you have calculated the distance between points in terms of matrix operations, you are ready to write an efficient program that leverages NumPy's optimized functions. In this code exercise, rather than using loops, you will write a function to compute Euclidean distances between sets of vectors using NumPy functions.</p>

<h3>Evaluation</h3>

<p><strong>You must complete this exercise in order to unlock the final project in this module. Your score on this assignment will not be included in the final grade calculation.</strong><p>

<p>You are expected to write code where you see <em># YOUR CODE HERE</em> within the cells of this notebook. Not all cells will be graded; code input cells followed by cells marked with <em>#Autograder test cell</em> will be graded. Upon submitting your work, the code you write at these designated positions will be assessed using an "autograder" that will run all test cells to assess your code. You will receive feedback from the autograder that will identify any errors in your code. Use this feedback to improve your code if you need to resubmit. Be sure not to change the names of any provided functions, classes, or variables within the existing code cells, as this will interfere with the autograder. Also, remember to execute all code cells sequentially, not just those you’ve edited, to ensure your code runs properly.</p>
    
<p>You can resubmit your work as many times as necessary before the submission deadline. If you experience difficulty or have questions about this exercise, use the Q&A discussion board to engage with your peers or seek assistance from the instructor.<p>

<p>Before starting your work, please review <a href="https://s3.amazonaws.com/ecornell/global/eCornellPlagiarismPolicy.pdf">eCornell's policy regarding plagiarism</a> (the presentation of someone else's work as your own without source credit).</p>

<h3>Submit Code for Autograder Feedback</h3>

<p>Once you have completed your work on this notebook, you will submit your code for autograder review. Follow these steps:</p>

<ol>
  <li><strong>Save your notebook.</strong></li>
  <li><strong>Mark as Completed —</strong> In the blue menu bar along the top of this code exercise window, you’ll see a menu item called <strong>Education</strong>. In the <strong>Education</strong> menu, click <strong>Mark as Completed</strong> to submit your code for autograder/instructor review. This process will take a moment and a progress bar will show you the status of your submission.</li>
	<li><strong>Review your results —</strong> Once your work is marked as complete, the results of the autograder will automatically be presented in a new tab within the code exercise window. You can click on the assessment name in this feedback window to see more details regarding specific feedback/errors in your code submission.</li>
  <li><strong>Repeat, if necessary —</strong> The Jupyter notebook will always remain accessible in the first tabbed window of the exercise. To reattempt the work, you will first need to click <strong>Mark as Uncompleted</strong> in the <strong>Education</strong> menu and then proceed to make edits to the notebook. Once you are ready to resubmit, follow steps one through three. You can repeat this procedure as many times as necessary.</li>
</ol>

<h2>Import NumPy and Check Python Version</h2>

First, you must import NumPy. Let's also check our version of Python. We've added the code for you for this first step.

In [3]:
import sys
import numpy as np # Numpy is Python's built in library for matrix operations.
from pylab import * 
sys.path.append('/home/codio/workspace/.guides/hf')
from helper import *
print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.6.8



<h2> Euclidean Distances in Python </h2>

<p>Many machine learning algorithms access their input data primarily through pairwise (Euclidean) distances, therefore it is important that we have a fast function that computes pairwise distances of input vectors.</p>
<p>Assume we have $n$ row data vectors $\mathbf{x_1}, \dots, \mathbf{x_n} \in \mathcal{R}^d$ and $m$ row vectors $\mathbf{z_1}, \dots, \mathbf{z_m} \in \mathcal{R}^d$. With these data vectors, let us define two matrices $X = \left[ \mathbf{x_1}^\top, \dots, \mathbf{x_n}^\top \right]^\top \in \mathcal{R}^{n \times d}$, where the $i^{th}$ row is vector $\mathbf{x_i}$, and $Z = \left[ \mathbf{z_1}^\top, \dots, \mathbf{z_m}^\top \right]^\top \in \mathcal{R}^{m \times d}$.We want a distance function that takes as input these two matrices $X$ and $Z$ and outputs a matrix $D\in{\cal R}^{n\times m}$, where 
	$$D_{ij}=\sqrt{(\mathbf{x_i}-\mathbf{z_j})(\mathbf{x_i}-\mathbf{z_j})^\top}.$$
</p>

A naïve implementation to compute pairwise distances may look like the code below:

In [4]:
def l2distanceSlow(X,Z=None):
    if Z is None:
        Z = X
    
    n, d = X.shape     # dimension of X
    m= Z.shape[0]   # dimension of Z
    D=np.zeros((n,m)) # allocate memory for the output matrix
    for i in range(n):     # loop over vectors in X
        for j in range(m): # loop over vectors in Z
            D[i,j]=0.0; 
            for k in range(d): # loop over dimensions
                D[i,j]=D[i,j]+(X[i,k]-Z[j,k])**2; # compute l2-distance between the ith and jth vector
            D[i,j]=np.sqrt(D[i,j]); # take square root
    return D

Please read through the code above carefully and make sure you understand it. It is perfectly correct and will produce the correct result... eventually. To see what is wrong, try running the <code>l2distanceSlow</code> code below on an extremely small matrix <code>X</code>.


In [5]:
X=np.random.rand(700,100)
print("Running the naive version, please wait...")
%time Dslow=l2distanceSlow(X)

Running the naive version, please wait...
CPU times: user 1min 2s, sys: 37.8 ms, total: 1min 2s
Wall time: 1min 2s


This code defines some random data in $X$ and computes the corresponding distance matrix $D$. The <em>%time</em> statement determines how long this code takes to run. This implementation is much too slow for such a simple operation on a small amount of data, and writing code like this to deal with matrices in this course will result in code that takes <strong>days</strong> to run.

<strong>As a general rule, you should avoid tight loops at all cost.</strong> As you will see in the remainder of this exercise, you can do much better by performing bulk matrix operations using the NumPy package, which calls highly optimized compiled code behind the scenes.





<h2> Efficient Programming with NumPy </h2>

<p>Although there is an execution overhead per line in Python, matrix operations are optimized and fast. In order to successfully program in this course, you need to free yourself from "for-loop" thinking and start thinking in terms of matrix operations. Python for scientific computing can be very fast if almost all the time is spent on a few heavy duty matrix operations. In this exercise, you will transform the function above into a few matrix operations <em>without any loops at all.</em> </p> 

<p>The key to efficient programming in Python for machine learning in general is to think about it in terms of mathematics and not in terms of loops. </p>

<h2>Exercises</h2>

<p>In the following three exercises, you'll take the steps necessary to implement the euclidean distance function without loops.</p>

<h3>Exercise 1: Inner-Product Matrix</h3>

<p>Show that the Inner-Product Matrix (Gram matrix) can be expressed in terms of pure matrix multiplication.

$$	G_{ij}=\mathbf{x}_i\mathbf{z}_j^\top $$

Once you are done with the derivation, implement the function <strong><code>innerproduct</code></strong>.</p>

In [6]:
def innerproduct(X,Z=None):
    '''
    function innerproduct(X,Z)
    
    Computes the inner-product matrix.
    Syntax:
    D=innerproduct(X,Z)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    
    Output:
    Matrix G of size nxm
    G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    
    call with only one input:
    innerproduct(X)=innerproduct(X,X)
    '''
    if Z is None: # case when there is only one input (X)
        Z=X;
        
    n, d = X.shape
    m, d = Z.shape
    z_transpose = Z.T
    G = X @ z_transpose
    return G
    
    

    # YOUR CODE HERE
    # raise NotImplementedError()

In [7]:
#Run this self-test cell to check your code

def innerprod_0():
    # test the output dimensions of innerproduct with one input matrix
    X = np.random.rand(700,10) # define 700 random inputs X
    test = (innerproduct(X).shape==700,700)    # check if inner-product matrix has dimension 700x700
    return test

def innerprod_1():
    # test the output dimensions of innerproduct with two matrices
    X = np.random.rand(700,10) # define 700 random inputs X
    Z = np.random.rand(200,10) # define 200 random inputs Z 
    test=(innerproduct(X,Z).shape ==(700,200)) # check if inner-product matrix has dimensions 700x200
    return test

def innerprod_2():
    X = np.random.rand(700,100) # define 700 random inputs X
    IP1 = innerproduct(X) # compute inner-product matrix with YOUR code
    IP2 = innerproduct_grader(X) # compute inner-product matrix with OUR code
    test = np.linalg.norm(IP1 - IP2) # compute the norm of the difference
    return test<1e-5 # this norm should be essentially 0

def innerprod_3():
    X = np.random.rand(700,100) # define 700 random inputs X
    Z = np.random.rand(300,100) # define 300 random inputs X
    IP1 = innerproduct(X,Z) # compute inner-product matrix with YOUR code
    IP2 = innerproduct_grader(X,Z) # compute inner-product matrix with OUR code
    test = np.linalg.norm(IP1 - IP2) # compute the norm of the difference
    return test<1e-5 # this norm should be essentially 0


runtest(innerprod_0,'innerprod_0 Dimensions with 1 Matrix')
runtest(innerprod_1,'innerprod_1 Dimensions with 2 Matrices')
runtest(innerprod_2,'innerprod_2 Correctness with 1 Matrix')
runtest(innerprod_3,'innerprod_3 Correctness with 2 Matrices')

Running Test: innerprod_0 Dimensions with 1 Matrix ... ✔ Passed!
Running Test: innerprod_1 Dimensions with 2 Matrices ... ✔ Passed!
Running Test: innerprod_2 Correctness with 1 Matrix ... ✔ Passed!
Running Test: innerprod_3 Correctness with 2 Matrices ... ✔ Passed!


In [8]:
# Autograder test cell - worth 1 point
# runs innerprod_1

In [9]:
# Autograder test cell - worth 1 Point
# runs innerprod_2

### Exercise 2: Implement `calculate_S` and `calculate_R`

Let us define two new matrices, $S, R \in \mathcal{R}^{n \times m}$
$$
S_ij = \mathbf{x}_i \mathbf{x}_i^\top, \ \ R_{ij} = \mathbf{z}_j \mathbf{z}_j^\top
$$

In the previous activity, we have shown that the _squared_-euclidean matrix $D^2 \in \mathcal{R}^{n \times m}$, defined as
$$
D_{ij}^2 = (\mathbf{x}_i - \mathbf{z}_j)(\mathbf{x}_i - \mathbf{z}_j)^\top,
$$
can be expressed as
$$
D^2 = S + R - 2G.
$$

Later in this exercise, you will implement `l2distance` to calculate $D$. But you will need $S$ and $R$, which you will implement now in **`calculate_S`** and **`calculate_R`** respectively. Ensure that your functions return $S$ and $R$ of size $n \times m$, as they will be added to $-2G$ to get $D^2$.

Think about what the $S$ and $R$ matrices look like. You will find that the values in each row of $S$ and the values in each column of $R$ do not change! This is also apparent when considering that $S_{ij} = \mathbf{x}_i \mathbf{x}_i^\top$ _for all $j$_; similar argument for $R_{ij} = \mathbf{z}_j \mathbf{z}_j^\top$ _for all $i$_.
$$
S = \begin{bmatrix}
\mathbf{x}_1 \mathbf{x}_1^\top & \mathbf{x}_1 \mathbf{x}_1^\top & \cdots & \mathbf{x}_1 \mathbf{x}_1^\top\\
\mathbf{x}_2 \mathbf{x}_2^\top & \mathbf{x}_2 \mathbf{x}_2^\top & \cdots & \mathbf{x}_2 \mathbf{x}_2^\top\\
\vdots & \vdots & \ddots & \vdots\\
\mathbf{x}_n \mathbf{x}_n^\top & \mathbf{x}_n \mathbf{x}_n^\top & \cdots & \mathbf{x}_n \mathbf{x}_n^\top\\
\end{bmatrix}, \ 
R = \begin{bmatrix}
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\vdots & \vdots & \ddots & \vdots\\
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\end{bmatrix}.
$$

Now you just need to figure out how to calculate $\mathbf{x}_i \mathbf{x}_i^\top$ and $\mathbf{z}_j \mathbf{z}_j^\top$ without loops. You might find the fact $\mathbf{a} \mathbf{a}^\top = \sum_{k=1}^d a_k^2$ and repeat function [`np.repeat`](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html) (and its `axis` parameter) useful.

In [75]:
def calculate_S(X, n, m):
    '''
    function calculate_S(X)
    
    Computes the S matrix.
    Syntax:
    S=calculate_S(X)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    n: number of rows in X
    m: output number of columns in S
    
    Output:
    Matrix S of size nxm
    S[i,j] is the inner-product between vectors X[i,:] and X[i,:]
    '''
    assert n == X.shape[0]
    S= np.sum(np.square(X), axis = 1)
    # print("This is np.sum result", S)
    S = S.reshape((n, 1)) @ np.ones((1,m))
    return S
    
    # YOUR CODE HERE
    # raise NotImplementedError()

In [113]:
X = X = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [114]:
Z = 2*X

In [115]:
Z

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

In [118]:
calculate_S(X, 3, 3)

array([[ 14.,  14.,  14.],
       [ 77.,  77.,  77.],
       [194., 194., 194.]])

In [119]:
innerproduct(X,Z)

array([[ 28,  64, 100],
       [ 64, 154, 244],
       [100, 244, 388]])

In [120]:
calculate_S(X, 3, 3) + calculate_R(Z, 3, 3) - 2*innerproduct(X,Z)

This is np.sum result [ 56 308 776]


array([[ 14., 194., 590.],
       [  5.,  77., 365.],
       [ 50.,  14., 194.]])

In [94]:
def calculate_R(Z, n, m):
    '''
    function calculate_R(Z)
    
    Computes the R matrix.
    Syntax:
    R=calculate_R(Z)
    Input:
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    n: output number of rows in Z
    m: number of rows in Z
    
    Output:
    Matrix R of size nxm
    R[i,j] is the inner-product between vectors Z[j,:] and Z[j,:]
    '''
    assert m == Z.shape[0]
    R= np.sum(np.square(Z), axis = 1)
    print("This is np.sum result", R)
    R = np.ones((n, 1)) @ R.reshape((1, m))
    return R
    
    # YOUR CODE HERE
    # raise NotImplementedError()

In [95]:
#Run this self-test cell to check your code

def calculate_S_dimensions():
    X = np.random.rand(700,100) # define random inputs
    Z = np.random.rand(800,100) # define random inputs
    n,d1=X.shape
    m,d2=Z.shape    
    S1 = calculate_S(X, n, m) # compute distances from your solutions
    o1,o2=S1.shape
    return (o1==n) and (o2==m)

def calculate_S_accuracy():
    X = np.random.rand(700,100) # define random inputs
    S1 = calculate_S(X, X.shape[0], 800) # compute distances from your solutions
    S2 = calculate_S_grader(X, X.shape[0], 800) #compute distance from ground truth
    test = np.linalg.norm(S1 - S2) # compare the two
    return test<1e-5 # difference should be small

def calculate_R_dimensions():
    X = np.random.rand(700,100) # define random inputs
    Z = np.random.rand(800,100) # define random inputs
    n,d1=X.shape
    m,d2=Z.shape    
    R1 = calculate_R(Z, n, m) # compute distances from your solutions
    o1,o2=R1.shape
    return (o1==n) and (o2==m)

def calculate_R_accuracy():
    Z = np.random.rand(800,100) # define random inputs
    R1 = calculate_R(Z, 700, Z.shape[0]) # compute distances from your solutions
    R2 = calculate_R_grader(Z, 700, Z.shape[0]) #compute distance from ground truth
    test = np.linalg.norm(R1 - R2) # compare the two
    return test<1e-5 # difference should be small

runtest(calculate_S_dimensions,'calculate_S_dimensions')
runtest(calculate_S_accuracy,'calculate_S_accuracy')
runtest(calculate_R_dimensions,'calculate_R_dimensions')
runtest(calculate_R_accuracy,'calculate_R_accuracy')

Running Test: calculate_S_dimensions ... ✔ Passed!
Running Test: calculate_S_accuracy ... ✔ Passed!
Running Test: calculate_R_dimensions ... This is np.sum result [32.9934887  32.53136987 33.68089943 31.5362549  32.57818681 35.51761641
 33.22186019 40.75199073 38.74878858 33.61573498 28.77422129 32.71273357
 38.02778434 30.37024264 29.21097739 30.37341595 33.08668608 34.62441208
 33.55126742 34.56914226 36.44618275 27.89920247 33.55207858 35.14969148
 34.4968579  31.8083273  30.79579591 29.40838864 27.81161637 31.00799184
 31.46570479 30.0873948  33.42717781 35.70286166 38.19893276 30.09396768
 37.29464271 29.70539251 30.70310033 32.36684075 38.91106384 31.43600391
 38.1798724  33.60361463 31.78067673 35.27453648 33.31990885 34.63334936
 33.02620306 32.05660921 36.44317897 34.78497775 31.23569854 35.75921681
 36.06797923 33.42821466 34.79420814 35.55607726 33.10299036 37.06873877
 35.62385336 33.81476917 33.57864582 35.61313042 34.06549414 36.76030242
 37.17210551 31.49043941 31.668144

<h3>Exercise 3: Implement <code>l2distance</code></h3>

$$
D^2_{ij}=(\mathbf{x}_i-\mathbf{z}_j)(\mathbf{x}_i-\mathbf{z}_j)^\top, \ D^2 = S + R - 2G
$$
    
<p> In this exercise, you will use the above formula to implement the function <strong><code>l2distance</code></strong>, which computes the Euclidean distance matrix $D$ without a single loop. </p>

<p><strong>Hint</strong>: Make sure that when you take the square root of the squared distance matrix, ensure that all entries are non-negative. Sometimes very small positive numbers can become negative due to numerical imprecision. Knowing that all distances must always be non-negative, you can simply overwrite all negative values as 0.0 to avoid unintended consequences </p>

In [121]:
def l2distance(X,Z=None):
    '''
    function D=l2distance(X,Z)
    
    Computes the Euclidean distance matrix.
    Syntax:
    D=l2distance(X,Z)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    
    Output:
    Matrix D of size nxm
    D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    
    call with only one input:
    l2distance(X)=l2distance(X,X)
    '''
    if Z is None:
        Z=X;

    n,d1=X.shape
    m,d2=Z.shape
    assert (d1==d2), "Dimensions of input vectors must match!"
    S = calculate_S(X, n, m)
    R = calculate_R(Z, n, m)
    G = innerproduct(X,Z)
    D_squared = S + R -2*G
    D_squared[D_squared < 0] = 0
    D = np.sqrt(D_squared)
    return D

    
    # YOUR CODE HERE
    #raise NotImplementedError()

In [122]:
#Run this self-test cell to check your code

def distance_accuracy(): 
    X = np.random.rand(700,100) # define random inputs
    D1 = l2distance(X) # compute distances from your solutions
    D2 = l2distance_grader(X) #compute distance from ground truth
    test = np.linalg.norm(D1 - D2) # compare the two
    return test<1e-5 # difference should be small

def distance_squareroot():  
    X = np.random.rand(700,100) # define random inputs
    D1 = l2distance(X) # compute distances from your solutions
    D2sq = l2distance_grader(X)**2 #compute distance from ground truth *but square them*
    test = np.linalg.norm(D1 - D2sq) # compare the two
    return test>1e-5 # difference should be big

def dimensions():
    X = np.random.rand(700,100) # define random inputs
    Z = np.random.rand(800,100) # define random inputs
    n,d1=X.shape
    m,d2=Z.shape    
    D1 = l2distance(X,Z) # compute distances from your solutions
    o1,o2=D1.shape
    return (o1==n) and (o2==m)

def matrix_dist_accuracy():
    X = np.random.rand(700,100)
    Z = np.random.rand(300,100)
    D1Z = l2distance(X,Z)
    D2Z = l2distance_grader(X,Z)
    test = np.linalg.norm(D1Z - D2Z)
    return test<1e-5

runtest(distance_accuracy,'distance_accuracy')
runtest(distance_squareroot,'distance_squareroot')
runtest(dimensions,'dimensions')
runtest(matrix_dist_accuracy,'matrix_dist_accuracy')

Running Test: distance_accuracy ... This is np.sum result [39.75768242 28.94595805 29.71321089 31.01318743 31.12127052 36.45484912
 32.81200866 32.31004862 33.10546817 36.40490194 29.55401885 29.82731341
 29.53726665 34.12617763 33.4895164  29.45755656 37.01799854 36.68156
 28.24871487 31.65806251 34.03169595 29.05995878 30.0771172  32.46330293
 34.07785227 36.03506469 35.90882361 33.43650585 33.16996253 35.61512393
 34.14216409 33.50296813 31.3090321  35.77116539 37.53359388 29.4784538
 37.19795844 28.37533314 34.64600316 37.42441482 35.88554906 37.77788496
 33.89923865 32.63521016 31.64372231 31.33163897 31.74193318 38.42487622
 36.51933078 34.00496333 35.54326047 32.67195442 30.44279183 31.07078923
 33.29001342 32.32573195 32.07495863 35.96096018 29.81993027 34.44961293
 37.98935895 30.36310927 30.07862286 29.27995549 28.78049248 34.13325964
 30.38776433 31.34508172 27.18010859 31.95722825 37.43295466 41.40663606
 33.7571325  33.98923928 41.24059958 30.02169138 31.61062024 31.032954

In [ ]:
# Autograder test cell - worth 1 Point
# runs distance_accuracy

In [124]:
l2distance(X,Z)

This is np.sum result [34.77867208 36.56469181 42.26943881 35.00440358 29.10777316 31.56932154
 27.62557799 28.69250361 36.87180973 33.11536716 33.36325858 39.64194643
 36.38006785 33.9050355  29.65639482 29.70904853 33.61627584 30.34690777
 33.27978632 28.28216985 34.25744752 30.08657509 33.47669818 30.82862443
 40.62817497 33.23621629 39.65458752 32.98588694 34.22774829 27.8890802
 34.63116077 34.72651767 35.41240701 35.48466107 37.41800255 38.7130464
 30.76190359 35.14969058 30.85109666 34.46050414 33.53009713 36.46440766
 38.47556142 31.82556079 30.8781435  35.63921489 27.31672444 31.5781833
 32.46678103 39.06985816 39.79196461 35.00885664 36.51961283 30.3372139
 29.13518787 33.5945501  38.08107575 28.73486026 32.06052138 33.67437574
 34.21003806 31.17811562 33.72528245 28.19199833 34.29829061 34.21773467
 32.17675471 33.38573727 31.82453269 35.65846835 32.69077539 36.50778263
 33.39343454 30.17518761 38.65891212 32.34421136 32.53258515 36.08953017
 33.15595781 32.98035808 30.78241

array([[4.17302565, 4.40260862, 3.84096668, ..., 3.89594048, 4.27220359,
        4.19077189],
       [4.15313817, 3.9330718 , 4.02251615, ..., 3.73525329, 4.44110588,
        4.20410446],
       [3.76582517, 3.93752425, 3.91220779, ..., 3.59546177, 3.93520863,
        3.94089607],
       ...,
       [3.43905477, 4.13886539, 3.91015864, ..., 3.79736863, 4.00644844,
        4.33770807],
       [3.92008669, 3.87907104, 4.32859808, ..., 4.12956023, 3.82545157,
        4.28106029],
       [4.32535635, 4.1601363 , 4.21273863, ..., 3.96900073, 4.15480849,
        4.08513342]])

In [ ]:
# Autograder test cell - worth 1 Point
# runs distance_squareroot

In [ ]:
# Autograder test cell - worth 1 Point
# runs dimensions

In [ ]:
# Autograder test cell - worth 1 Point
# runs matrix_dist_accuracy

Let's now compare the speed of your l2-distance function against the previous naïve implementation:

In [123]:
import time
current_time = lambda: int(round(time.time() * 1000))

X=np.random.rand(700,100)
Z=np.random.rand(300,100)

print("Running the naïve version...")
before = current_time()
Dslow=l2distanceSlow(X)
after = current_time()
t_slow = after - before
print("{:2.0f} ms".format(t_slow))

print("Running the vectorized version...")
before = current_time()
Dfast=l2distance(X)
after = current_time()
t_fast = after - before
print("{:2.0f} ms".format(t_fast))


speedup = t_slow / t_fast
print("The two methods should deviate by very little: {:05.6f}".format(norm(Dfast-Dslow)))
print("but your NumPy code was {:05.2f} times faster!".format(speedup))

Running the naïve version...
60709 ms
Running the vectorized version...
This is np.sum result [32.33132266 31.56702955 33.19103109 33.20520348 37.57242308 37.76814936
 30.09229827 38.58756536 33.24873415 32.24040133 33.20073069 32.55429649
 39.55905687 34.81789686 35.86434385 33.18487917 33.6318251  32.0847548
 32.54765673 31.05246789 36.27078315 34.68534529 39.70601917 28.76286399
 37.55687463 32.13260057 27.6432853  32.69371783 38.08281605 34.95729367
 31.20527047 33.87873653 35.63990826 35.42526257 36.829972   33.23175003
 27.55334417 34.25575526 34.28037403 35.08337786 37.06812658 33.13605832
 32.95416794 35.00908506 31.92863101 30.40923325 31.52355983 29.16336781
 34.45256056 34.59726287 32.881129   35.81444026 31.37316143 30.48946169
 33.92955721 31.79501693 33.49398606 33.72811105 38.05314972 36.93764955
 37.30407841 33.35433008 33.59341285 32.78865443 33.8076734  27.89316201
 29.45161903 30.00934307 35.34667078 31.56977813 37.30994391 33.57424354
 34.60118345 33.49212335 32.822

How much faster is your code now? With this implementation, you should easily be able to compute the distances between <strong>many more</strong> vectors. It should be clear now, even for small datasets, that the for-loop based implementation could take several days or even weeks to perform basic operations that take seconds or minutes with well-written NumPy code.